# Trade Area Modelling with Huff Model

The project objective is to identify the most optimal location for a retail chain to open a new store in Montreal.

The dataset contains 3 sheets, the first sheet 'Distance' contains information about the retail store's customer community based in Montreal and their distance (in kilometres) to the nearest existing store. The 'Census' sheet contains data about the number of households in each community and their average expenditure on grocery. The 'Stores' sheet contains data about the attractiveness attributes - size, parking space, highways, traffic, accessibility and design.

In [1]:
import pandas as pd
import numpy as np

In [2]:
#Importing store's data containing customer community names and their distance to the existing stores (in km)
xls = pd.ExcelFile(r"C:\Users\praja\OneDrive\Desktop\1 Trade Area Modelling\trade.xlsx')

In [3]:
# Verifying that store's attractiveness attributes have been loaded
xls.sheet_names

['Distance', 'Census', 'stores']

In [6]:
#Loading distance data into datatframe and assigning customer community as index column
distance = xls.parse(0, index_col=0)

In [7]:
print(distance)

                    Rosmere  saint-rose  Laval de Rapides  Boisbraind
customer community                                                   
Blainville               30          70               130          40
Boisbrand                20          20               110           1
Saint therese            20          40               130          20
Fabre villee             50          20                40          40
Laval de Rapides         80          70                 1          80
Sainte Dorothee         100          70                40         100
Carterville             120          80                20         100
bois des filion          30          70               120          80
Deux Montagnes          150         100                90         100


In [8]:
#Loading the 2nd Excel sheet census into a dataframe
census = xls.parse(1, index_col=0)

In [9]:
print(census)

                    Households  Expenditure grocery
customer community                                 
Blainville                4000                  800
Boisbrand                 8000                 1000
Saint therese            12000                 1100
Fabre villee              6000                  700
Laval de Rapides          1000                 1200
Sainte Dorothee           7000                  500
Carterville               9000                  700
bois des filion           5000                  900
Deux Montagnes            4000                 1400


In [10]:
# Creating a new column 'Total Expenditure'

census['Total Expenditure'] = census['Households']*census['Expenditure grocery']

In [14]:
# Loading 3rd Excel sheet 'stores' into a dataframe
attractiveness = xls.parse(2, index_col=0)

In [15]:
attractiveness

,Size,Parking spaces,Highways,Traffic,Accessibility,Design
customer community,,,,,,
Rosmere,1000,200,4,4,10,7
saint-rose,1100,150,2,8,7,7
Laval de Rapides,800,120,3,2,8,6
Boisbraind,1300,220,5,6,5,10


In [16]:
stores = np.unique(attractiveness.index)

In [17]:
communities = np.unique(distance.index)

In [21]:
# Creating a tuple that contains every store and every community
# x = community, y = stores
keys = [(x,y) for x in communities for y in stores]

In [22]:
keys # Now every customer community type is in every store

[('Blainville', 'Boisbraind'),
 ('Blainville', 'Laval de Rapides'),
 ('Blainville', 'Rosmere'),
 ('Blainville', 'saint-rose'),
 ('Boisbrand', 'Boisbraind'),
 ('Boisbrand', 'Laval de Rapides'),
 ('Boisbrand', 'Rosmere'),
 ('Boisbrand', 'saint-rose'),
 ('Carterville', 'Boisbraind'),
 ('Carterville', 'Laval de Rapides'),
 ('Carterville', 'Rosmere'),
 ('Carterville', 'saint-rose'),
 ('Deux Montagnes', 'Boisbraind'),
 ('Deux Montagnes', 'Laval de Rapides'),
 ('Deux Montagnes', 'Rosmere'),
 ('Deux Montagnes', 'saint-rose'),
 ('Fabre villee', 'Boisbraind'),
 ('Fabre villee', 'Laval de Rapides'),
 ('Fabre villee', 'Rosmere'),
 ('Fabre villee', 'saint-rose'),
 ('Laval de Rapides', 'Boisbraind'),
 ('Laval de Rapides', 'Laval de Rapides'),
 ('Laval de Rapides', 'Rosmere'),
 ('Laval de Rapides', 'saint-rose'),
 ('Saint therese', 'Boisbraind'),
 ('Saint therese', 'Laval de Rapides'),
 ('Saint therese', 'Rosmere'),
 ('Saint therese', 'saint-rose'),
 ('Sainte Dorothee', 'Boisbraind'),
 ('Sainte Dorot

## Scaling the attractiveness attributes

In [23]:
scaled = (attractiveness-attractiveness.min())/(attractiveness.max()-attractiveness.min())

In [24]:
print(scaled)

,Size,Parking spaces,Highways,Traffic,Accessibility,Design
customer community,,,,,,
Rosmere,0.4,0.8,0.666667,0.333333,1.0,0.25
saint-rose,0.6,0.3,0.000000,1.000000,0.4,0.25
Laval de Rapides,0.0,0.0,0.333333,0.000000,0.6,0.00
Boisbraind,1.0,1.0,1.000000,0.666667,0.0,1.00


In [25]:
# Adding the scaled attributes and storing in a new column
scaled['Attractiveness'] = (scaled['Size'] + scaled['Parking spaces'] + scaled['Highways'] + scaled['Traffic'] + scaled['Accessibility'] + scaled['Design'])

In [26]:
print(scaled)

                    Size  Parking spaces  Highways   Traffic  Accessibility  \
customer community                                                            
Rosmere              0.4             0.8  0.666667  0.333333            1.0   
saint-rose           0.6             0.3  0.000000  1.000000            0.4   
Laval de Rapides     0.0             0.0  0.333333  0.000000            0.6   
Boisbraind           1.0             1.0  1.000000  0.666667            0.0   

                    Design  Attractiveness  
customer community                          
Rosmere               0.25        3.450000  
saint-rose            0.25        2.550000  
Laval de Rapides      0.00        0.933333  
Boisbraind            1.00        4.666667  


Boisbraind has the highest Attractiveness score.

## Calculating the numerator for every community and every store

In [27]:
numerator = {}

In [28]:
# Every key's 0 element is community and 1st element is store

for key in keys:
    numerator[key] = scaled.loc[key[1], 'Attractiveness']/distance.loc[key[0],key[1]]**2 

In [29]:
print(numerator) # Now we get numerator for every community and every store

{('Blainville', 'Boisbraind'): 0.0029166666666666664, ('Blainville', 'Laval de Rapides'): 5.522682445759369e-05, ('Blainville', 'Rosmere'): 0.0038333333333333336, ('Blainville', 'saint-rose'): 0.0005204081632653061, ('Boisbrand', 'Boisbraind'): 4.666666666666666, ('Boisbrand', 'Laval de Rapides'): 7.713498622589531e-05, ('Boisbrand', 'Rosmere'): 0.008625, ('Boisbrand', 'saint-rose'): 0.006375, ('Carterville', 'Boisbraind'): 0.0004666666666666666, ('Carterville', 'Laval de Rapides'): 0.0023333333333333335, ('Carterville', 'Rosmere'): 0.00023958333333333335, ('Carterville', 'saint-rose'): 0.0003984375, ('Deux Montagnes', 'Boisbraind'): 0.0004666666666666666, ('Deux Montagnes', 'Laval de Rapides'): 0.00011522633744855967, ('Deux Montagnes', 'Rosmere'): 0.00015333333333333334, ('Deux Montagnes', 'saint-rose'): 0.00025499999999999996, ('Fabre villee', 'Boisbraind'): 0.0029166666666666664, ('Fabre villee', 'Laval de Rapides'): 0.0005833333333333334, ('Fabre villee', 'Rosmere'): 0.00138000000

## Calculating the probability of customers visiting the stores

In [30]:
# Creating an empty Probability dictionary
Pijs = {}

In [31]:
# Viewing the keys (customers) and their values (stores) for reference
numerator.items()

dict_items([(('Blainville', 'Boisbraind'), 0.0029166666666666664), (('Blainville', 'Laval de Rapides'), 5.522682445759369e-05), (('Blainville', 'Rosmere'), 0.0038333333333333336), (('Blainville', 'saint-rose'), 0.0005204081632653061), (('Boisbrand', 'Boisbraind'), 4.666666666666666), (('Boisbrand', 'Laval de Rapides'), 7.713498622589531e-05), (('Boisbrand', 'Rosmere'), 0.008625), (('Boisbrand', 'saint-rose'), 0.006375), (('Carterville', 'Boisbraind'), 0.0004666666666666666), (('Carterville', 'Laval de Rapides'), 0.0023333333333333335), (('Carterville', 'Rosmere'), 0.00023958333333333335), (('Carterville', 'saint-rose'), 0.0003984375), (('Deux Montagnes', 'Boisbraind'), 0.0004666666666666666), (('Deux Montagnes', 'Laval de Rapides'), 0.00011522633744855967), (('Deux Montagnes', 'Rosmere'), 0.00015333333333333334), (('Deux Montagnes', 'saint-rose'), 0.00025499999999999996), (('Fabre villee', 'Boisbraind'), 0.0029166666666666664), (('Fabre villee', 'Laval de Rapides'), 0.00058333333333333

In [33]:
# Calculating the probablity of a customer (key) visiting a store
for key in keys:
    Pijs[key] = numerator[key]/sum([v for k,v in numerator.items() if k[0]==key[0]])

In [34]:
print(Pijs) 

{('Blainville', 'Boisbraind'): 0.3981452354034477, ('Blainville', 'Laval de Rapides'): 0.00753884469402978, ('Blainville', 'Rosmere'): 0.5232765951016742, ('Blainville', 'saint-rose'): 0.0710393248008484, ('Boisbrand', 'Boisbraind'): 0.9967795899081655, ('Boisbrand', 'Laval de Rapides'): 1.6475695700961416e-05, ('Boisbrand', 'Rosmere'): 0.001842262277776699, ('Boisbrand', 'saint-rose'): 0.0013616721183566905, ('Carterville', 'Boisbraind'): 0.13573700954400847, ('Carterville', 'Laval de Rapides'): 0.6786850477200425, ('Carterville', 'Rosmere'): 0.0696864111498258, ('Carterville', 'saint-rose'): 0.11589153158612331, ('Deux Montagnes', 'Boisbraind'): 0.47127272727272723, ('Deux Montagnes', 'Laval de Rapides'): 0.11636363636363636, ('Deux Montagnes', 'Rosmere'): 0.15484675324675326, ('Deux Montagnes', 'saint-rose'): 0.2575168831168831, ('Fabre villee', 'Boisbraind'): 0.25914408411076556, ('Fabre villee', 'Laval de Rapides'): 0.05182881682215312, ('Fabre villee', 'Rosmere'): 0.1226121723678

In the above output, the numeric values are the probability of the customer visiting the area. For example, the probablity that a customer from Blainville will go to a Boisbrand store is 0.39.

## Calculating the expected sales for every customer (key)

In [35]:
exp_key = {}

In [36]:
print(census)

                    Households  Expenditure grocery  Total Expenditure
customer community                                                    
Blainville                4000                  800            3200000
Boisbrand                 8000                 1000            8000000
Saint therese            12000                 1100           13200000
Fabre villee              6000                  700            4200000
Laval de Rapides          1000                 1200            1200000
Sainte Dorothee           7000                  500            3500000
Carterville               9000                  700            6300000
bois des filion           5000                  900            4500000
Deux Montagnes            4000                 1400            5600000


In [37]:
for key in keys:
    exp_key[key] = Pijs[key]*census.loc[key[0], 'Total Expenditure']

In [38]:
print(exp_key)

{('Blainville', 'Boisbraind'): 1274064.7532910327, ('Blainville', 'Laval de Rapides'): 24124.303020895295, ('Blainville', 'Rosmere'): 1674485.1043253576, ('Blainville', 'saint-rose'): 227325.83936271488, ('Boisbrand', 'Boisbraind'): 7974236.719265324, ('Boisbrand', 'Laval de Rapides'): 131.80556560769134, ('Boisbrand', 'Rosmere'): 14738.098222213594, ('Boisbrand', 'saint-rose'): 10893.376946853525, ('Carterville', 'Boisbraind'): 855143.1601272534, ('Carterville', 'Laval de Rapides'): 4275715.800636267, ('Carterville', 'Rosmere'): 439024.3902439025, ('Carterville', 'saint-rose'): 730116.6489925769, ('Deux Montagnes', 'Boisbraind'): 2639127.2727272725, ('Deux Montagnes', 'Laval de Rapides'): 651636.3636363636, ('Deux Montagnes', 'Rosmere'): 867141.8181818182, ('Deux Montagnes', 'saint-rose'): 1442094.5454545452, ('Fabre villee', 'Boisbraind'): 1088405.1532652155, ('Fabre villee', 'Laval de Rapides'): 217681.03065304313, ('Fabre villee', 'Rosmere'): 514971.12394491344, ('Fabre villee', 's

The above output displays the expected sales derieved from a community visting the store location. For example, Boisbraind makes a sale of around 1 million dollars from a customer community based in Blainville.

## Calculating the expected sales for each store

In [40]:
exp_store = {}

In [41]:
for store in stores:
    exp_store[store] = sum([v for k,v in exp_key.items() if k[1]==store])

In [42]:
print(exp_store)

{'Boisbraind': 22341001.597888175, 'Laval de Rapides': 7522796.403432679, 'Rosmere': 12681462.441282213, 'saint-rose': 7154739.557396932}


# Conclusion: Boisbrand is the winner!

## From the above output, the Boisbraind store displays the highest sales of around 22 million dollars. Therefore, Boisbraind would be the best choice to open a new store and also to maximize expected sales.